# OCR (_Optical Character Recognition_)

## Objective:

    - The main objective of this project is to extract the customer information from the monthly rechnung (bill).
    - Initially the "Pdf" is converted into the "Image" and then 'opencv' library is used for adjusting the color and thershold.
        + While converting the pdf to image. download and mention the 'Poppler' location
    - 'Tesseract' library is used for converting the image to string and data
        + download and mention the tesseract location
    - 'Regex' is utilized here for extracting the useful information like all details.
    - Then a dataframe is created for storing all details and further dataframe is stored as ".csv" file.
    - New Database connection is established for storing the .csv files and new tables are created for storing the image files
    - In this project postgresql database is used.


### 1. Importing and installing all libraries

In [1]:
%pip install pdf2image
%pip install tesseract

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2 as cv;
import pandas as pd;
import numpy as np;
import pytesseract;
import re;
from PIL import Image

### 2. Converting the pdf to image

In [3]:
from pdf2image import convert_from_path
images = convert_from_path('Sample_test.pdf', poppler_path= r'E:\integrify\Notebooks\Pytorch\poppler\Release-21.11.0-0\poppler-21.11.0\Library\bin')

### 3. Storing the images in the new folder

In [4]:
Label = 'test_samples'
if not os.path.exists(Label):
    os.mkdir(Label)

In [5]:
os.chdir('test_samples')
for i, image in enumerate(images):
    test_cases = 'image'+str(i)+'.jpg'
    image.save(test_cases, "JPEG")

### 4. Preprocessing the image for extracting the information

In [38]:
from pytesseract import Output
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe' #important part
img_ = cv.imread('image0.jpg')
image = cv.resize(img_, (1654,2339)) # size is original 
img = image.copy()
mask = np.zeros(image.shape, dtype=np.uint8)
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY) #convert an image from one color space to another
thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
# Filter for ROI using contour area and aspect ratio
countour = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
countour = countour[0] if len(countour) == 2 else countour[1]
#-----------------------------------------------------------#
""" Contours are defined as the line joining all the points along the boundary of an image that are having the same intensity. 
    Contours come handy in shape analysis, finding the size of the object of interest, and object detection.
    OpenCV has findContour() function that helps in extracting the contours from the image. 
    It works best on binary images, so we should first apply thresholding techniques, Sobel edges, etc.
   """
#-----------------------------------------------------------#
for c in countour:
    area = cv.contourArea(c)
    peri = cv.arcLength(c, True)
    approx = cv.approxPolyDP(c, 0.05 * peri, True) #Douglas-Peucker algorithm
    x,y,w,h = cv.boundingRect(approx)
    aspect_ratio = w / float(h)
    if area > 2000 and aspect_ratio > .5:
        mask[y:y+h, x:x+w] = image[y:y+h, x:x+w]
h, w, c = img.shape
boxes = pytesseract.image_to_boxes(img)
#------------------------------------------------------------#
for b in boxes.splitlines():
    b = b.split(' ')
    img = cv.rectangle(img, (int(b[1]), h - int(b[2])), (int(b[3]), h - int(b[4])), (0, 255, 0), 2)
d = pytesseract.image_to_data(img, output_type= Output.DICT)
n_boxes = len(d['text'])
#------------------------------------------------------------#
for i in range(n_boxes):
    if int(float(d['conf'][i])) > 60:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img2 = cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv.imshow('img', img)
cv.imshow('img2', img2)
# Perfrom OCR with Pytesseract
data = pytesseract.image_to_string(mask, lang='eng', config='--psm 6')
print(data)
cv.imshow('thresh', thresh)
cv.imshow('mask', mask)
cv.waitKey(0)

eine freenet Marke
Rechnungsdatum: 28.10.2021
Rechnungsnr.: M21067604209
Kundennummer: 37382491
Haben Sie Fragen zur
Rechnung? www.md.de/fa:
Post: mobilcom-debitel GmbH » 99076 Erfurt
mobilcom-debitel Kundenservice
Telefon: 040/55 55 41 000
mobilcom-debitel Kundenservice
Technik
Telefon: 0900/10 22 24 0
Herr € 2,49/Anruf, nur aus dem dt. Festnetz
Nikhil Raj Mule empichbar
Karl-Marx-Allee 3 Winenae
07747 Jena
Ihre mobilcom-debitel Rechnung Seite 1 von 2
Rechnungsbetrag netto 16,7984 €
USt.-Betrag (19%) 3,19€
Rechnungsbetrag gesamt 19,99 €
Die Begleichung der Rechnung erfolgt am 04.11.2021 im Lastschriftverfahren mit der Mandatsreferenz-Nummer
MC-37382491-00000001 von dem Konto: IBAN DE77**************9200.
Warnung vor SMS-Betrug: Aktuell zirkulieren in Deutschland auf den ersten Blick harmlos wirkende SMS, die z. B. auf
den Versand eines Paketes hinweisen und zur Uberpriifung aufrufen. Der in den SMS enthaltene Link sollte auf keinen Fall
angeklickt werden! Die Offnung des Links kann zu

-1

### 5. Extracting the customer information

In [9]:
#positive lookbehind
data_ = re.findall(r"(?<=Rufnummer\s)\d+\s\/\s\d+|(?<=nummer\:\s)\d+|(?<=netz\:\s)\w+|(?<=rif\:\s)\w+\s\w+\s\w+\s\wB|(?<=nehmer\:\s)\w+\s\w+\s\w+|(?<=Teilnehmer\:\s)\w+\s\w+\s\w+",data)
data_

['37382491',
 '0176 / 47344855',
 '224983810',
 'Telekom',
 'green LTE 6 GB',
 'Nikhil Raj Mule',
 '0176 / 47344855']

### 6. Creating and saving the dataframe as .CSV file

In [10]:
phone_bill_data = pd.DataFrame(data= { 'Kundennummer': [data_[0]],'Handy_nummer': [data_[1]], 'Vertragsnummer': [data_[2]], 
                    'Mobilfunknetz': [data_[3]], 'Tarif': [data_[4]], 'Teilnehmer': [data_[5]]})
phone_bill_data

,Kundennummer,Handy_nummer,Vertragsnummer,Mobilfunknetz,Tarif,Teilnehmer
0,37382491,0176 / 47344855,224983810,Telekom,green LTE 6 GB,Nikhil Raj Mule


In [12]:
phone_bill_data.to_csv('OCR_bills_data.csv')

### 7. Connecting to the database for storing the .csv file

In [13]:
param_dict={
    "host":"localhost",
    "database":"phone_bills",
    "user":"***",
    "password": "****"
}

In [14]:
%pip install psycopg2

In [15]:
from sqlalchemy.engine import create_engine
import psycopg2

In [16]:
connect = "postgresql://%s:%s@%s:5432/%s" %(param_dict["user"], param_dict["password"], param_dict["host"], param_dict["database"])

In [17]:
engine = create_engine(connect)
cur = engine.connect()

### 8. Saving the .csv file to the new database

In [18]:
phone_bill_data.to_sql("Phone_bills", cur)

### 9. Saving the image into the database

In [20]:
import psycopg2

# Connect to your postgres DB
conn = psycopg2.connect("dbname=phone_bills user=*** password=**** host=localhost")
curr = conn.cursor()

# Execute a query
curr.execute('select * from "Phone_bills"')

# Retrieve query results
curr.fetchall()

[(0,
  '37382491',
  '0176 / 47344855',
  '224983810',
  'Telekom',
  'green LTE 6 GB',
  'Nikhil Raj Mule')]

### 10. Creating the new table for storing the images

In [22]:
def create_tables():
    """ create tables in the PostgreSQL database"""
    commands = (
        """ CREATE TABLE images_data (
                serial_number SERIAL PRIMARY KEY,
                file_extension VARCHAR(5) NOT NULL,
                images BYTEA NOT NULL)
        """)
    conn = None
    try:
        # read the connection parameters
        # connect to the PostgreSQL server
        conn = psycopg2.connect("dbname=phone_bills user=*** password=**** host=localhost")
        cur = conn.cursor()
        # create table one by one
        cur.execute(commands)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

if __name__ == '__main__':
    create_tables()

### 11. Finally adding the image to the table

In [25]:
def write_blob(serial_number, path_to_file, file_extension):
    """ insert a BLOB into a table """
    conn = None
    try:
        # read data from a picture
        drawing = open(path_to_file,'rb').read()
        # read database configuration
        conn = psycopg2.connect("dbname=phone_bills user=*** password=**** host=localhost")
        # connect to the PostgresQL database
        # create a new cursor object
        cur = conn.cursor()
        # execute the INSERT statement
        cur.execute("INSERT INTO images_data(serial_number,file_extension, images)" +
                    "VALUES(%s,%s,%s)",
                    (serial_number, file_extension, psycopg2.Binary(drawing)))
        # commit the changes to the database
        conn.commit()
        # close the communication with the PostgresQL database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [26]:
write_blob(1,'image0.jpg', 'jpg')

### 12. Checking the image loaction

In [27]:
def read_blob(path_to_dir):
    """ read BLOB data from a table """
    conn = None
    try:
        # read database configuration
        # connect to the PostgresQL database
        conn = psycopg2.connect("dbname=phone_bills user=*** password=**** host=localhost")
        # connect to the PostgresQL database
        # create a new cursor object
        cur = conn.cursor()
        # execute the SELECT statement
        cur.execute(""" SELECT *
                        FROM images_data
                        """)

        blob = cur.fetchone()
        print(blob)
        open(path_to_dir +'.' + blob[1], 'wb').write(blob[2])
        # close the communication with the PostgresQL database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [28]:
read_blob("image0")

(1, 'jpg', <memory at 0x000001B4C5406580>)
